# 0. Magic Commands


In [8]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import Library

In [9]:
from fastai import *
from fastai.vision.all import *
from fastai.metrics import accuracy
from fastai.vision.augment import get_transforms

ImportError: cannot import name 'get_transforms' from 'fastai.vision.augment' (C:\Users\PP\anaconda3\envs\CallmeEnv\lib\site-packages\fastai\vision\augment.py)

# 2. ข้อมูล

ใช้ชุดข้อมูล Dataset [MNIST](http://yann.lecun.com/exdb/mnist/) ตัวเลขอารบิคเขียนด้วยลายมือ

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

# 3. เตรียมข้อมูล



In [ ]:
batchsize = 64
np.random.seed(55)
transform = get_transforms(do_flip=False)

databunch = ImageDataBunch.from_folder(path, train='training', 
                                       valid_pct=0.2, size=28, 
                                       ds_tfms=transform, bs=batchsize, 
                                       num_workers=8).normalize()

# 4. สร้างโมเดล

เพื่อความสะดวก เราจะไปสร้างก่อน fit แต่ละแบบ

* เคสนี้เราจะใช้โมเดล models.vgg16_bn คือ โมเดลชื่อว่า VGGNet มี 16 Layers จากปี 2014 ซึ่งเป็นโมเดลที่เก่ากว่า เก่งน้อยกว่า Resnet จะได้เห็นกราฟ Loss ได้ชัดขึ้น
* pretrained=False หมายถึง เราจะเริ่มเทรนตั้งแต่ต้น ไม่เอา Weight ที่เคยเทรนกับ ImageNet มาแล้วมาใช้

In [ ]:
# learner = cnn_learner(databunch, models.vgg16_bn, 
#                       pretrained=False, 
#                       metrics=accuracy, callback_fns=ShowGraph)

# 5. เริ่มต้นเทรนโมเดล

ปกติ Learning Rate จะมีค่า Default ประมาณ 3e-3 หรือ 0.003 เราจะลองเทรนด้วย Learning Rate สูงมาก และต่ำมาก เปรียบเทียบกับ [Image Classification ep.3](https://www.bualabs.com/archives/509/mnist-fastai-resnet18-image-classification-ep-3/) ดูว่าจะเป็นอย่างไร

## ลองเทรนด้วย Learning Rate สูง ๆ เช่น 0.1

In [ ]:
learner = cnn_learner(databunch, models.vgg16_bn, 
                      pretrained=False, 
                      metrics=accuracy, callback_fns=ShowGraph)
learner.fit(10, lr=0.1)

Weight ของ Model ถูกลบด้วยค่าที่ใหญ่เกินไป ซ้ำ ๆ เหมือนถูกระเบิดออก ทำให้ Loss กระเด้งไปมา ไม่ลงสู่จุดต่ำสุด 

บางที Weight ระเบิดมากเกินกว่าที่ตัวเลขในคอมพิวเตอร์ จะรับได้ จะกลายเป็นค่า NaN หรือ Not a Number ต้อง Restart และเทรนใหม่ตั้งแต่ต้น

## ลองเทรนด้วย Learning Rate ต่ำ ๆ เช่น 0.000001

In [ ]:
learner = cnn_learner(databunch, models.vgg16_bn, 
                      pretrained=False, 
                      metrics=accuracy, callback_fns=ShowGraph)
learner.fit(8, lr=0.000001)

Weight ของ Model ถูกลบด้วยค่าที่เล็กเกินไป ทำให้ Loss ไม่ขยับไปไหน 

## ลองเทรนด้วย Epoch น้อยเกินไป หรือ เทรนสั้นเกินไป

In [ ]:
learner = cnn_learner(databunch, models.vgg16_bn, 
                      pretrained=False, 
                      metrics=accuracy, callback_fns=ShowGraph)
learner.fit(1, lr=0.0003)

เทรนสั้นไป ยังไม่ได้อัพเดท Weight สักเท่าไร ทำให้ Loss ไม่ขยับไปไหน 

## ลองเทรนด้วย Epoch เยอะเกินไป หรือ เทรนนานเกินไป

In [ ]:
learner = cnn_learner(databunch, models.vgg16_bn, 
                      pretrained=False, 
                      metrics=accuracy, callback_fns=ShowGraph)
learner.fit(30, lr=0.0003)

เทรนนานเกินไป ผลลัพธ์ก็ไม่ได้ดีขึ้น บางทีอาจจะทำให้ Overfit ซึ่งไว้เราจะอธิบายต่อไป

# Credit 

* [FastAI: Practical Deep Learning for Coders, v3 - Lesson 2](https://course.fast.ai/videos/?lesson=2)